In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# -------------------------------------------------------------
# 1. Define hyperparameters
# -------------------------------------------------------------
IMG_WIDTH = 224
IMG_HEIGHT = 224
NUM_CLASSES = 7         # Number of emotion categories
BATCH_SIZE = 32         # Try different batch sizes (e.g., 16, 32, 64)
EPOCHS_STAGE1 = 10      # Number of epochs for stage 1 (freeze backbone)
EPOCHS_STAGE2 = 10      # Number of epochs for stage 2 (fine-tuning)
LEARNING_RATE_STAGE1 = 1e-4
LEARNING_RATE_STAGE2 = 1e-5

# Dataset directory structure (adjust based on actual data location)
TRAIN_DIR = "./archive/train"
VAL_DIR = "./archive/test"  # Or "./archive/val" depending on dataset setup

# -------------------------------------------------------------
# 2. Data augmentation and generators
# -------------------------------------------------------------
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=10,            # Small rotation
    width_shift_range=0.1,        # Horizontal shift
    height_shift_range=0.1,       # Vertical shift
    brightness_range=(0.9, 1.1),  # Adjust brightness
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    directory=TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    directory=VAL_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# -------------------------------------------------------------
# 3. Build pre-trained model (MobileNetV2)
# -------------------------------------------------------------
base_model = MobileNetV2(
    weights='imagenet',       # Use ImageNet pre-trained weights
    include_top=False,        # Exclude the top classification layer
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# Freeze all convolutional layers (only train the custom classification head in stage 1)
for layer in base_model.layers:
    layer.trainable = False

# -------------------------------------------------------------
# 4. Build custom classification head
# -------------------------------------------------------------
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

# -------------------------------------------------------------
# 5. Compile the model - Using Precision as a metric
# -------------------------------------------------------------
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE_STAGE1),
    loss='categorical_crossentropy',
    metrics=[tf.keras.metrics.Precision(name='precision')]
)

model.summary()

# -------------------------------------------------------------
# 6. Define callbacks
# -------------------------------------------------------------
callbacks = [
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,         # Reduce learning rate factor
        patience=3,         # Reduce LR after 3 epochs with no improvement
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=5,         # Stop training after 5 epochs with no improvement
        restore_best_weights=True,
        verbose=1
    )
]

# -------------------------------------------------------------
# 7. Stage 1 training (train only the classification head)
# -------------------------------------------------------------
print("=== Stage 1: Training top layers (base_model frozen) ===")
history_stage1 = model.fit(
    train_generator,
    epochs=EPOCHS_STAGE1,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

# -------------------------------------------------------------
# 8. Stage 2 fine-tuning - Unfreeze last 20 layers
# -------------------------------------------------------------
print("=== Stage 2: Fine-tuning some layers of base_model ===")
# Unfreeze the last 20 layers
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Recompile the model with a lower learning rate and keep Precision as the metric
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE_STAGE2),
    loss='categorical_crossentropy',
    metrics=[tf.keras.metrics.Precision(name='precision')]
)

history_stage2 = model.fit(
    train_generator,
    epochs=EPOCHS_STAGE2,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

# -------------------------------------------------------------
# 9. Evaluate and save the model
# -------------------------------------------------------------
test_loss, test_precision = model.evaluate(val_generator, verbose=1)
print(f"Final Test Loss: {test_loss:.4f} | Test Precision: {test_precision:.4f}")

model.save("emotion_recognition_transfer_precision.h5")
print("Model saved as emotion_recognition_transfer_precision.h5")